In [75]:
import tensorflow as tf

import keras
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

import pandas as pd
import numpy as np

import sklearn
from sklearn.preprocessing import Binarizer

'''
Tensorflow Image Data import needs tf-nightly (nightly updated build) package: 
pip install tf-nightly
'''

'\nTensorflow Image Data import needs tf-nightly (nightly updated build) package: \npip install tf-nightly\n'

In [77]:
#### Data Preparation ####

In [78]:
y_train = pd.read_csv('./cnn_input/y_train.csv')
y_test = pd.read_csv('./cnn_input/y_test.csv')

In [79]:
y_train = y_train.values
y_test = y_test.values

In [80]:
sklearn.preprocessing.Binarizer(threshold=0.0, copy=False)

y_train = Binarizer().fit_transform(y_train.reshape(1, -1))[0]
y_test = Binarizer().fit_transform(y_test.reshape(1, -1))[0]

In [81]:
len(y_train)

320228

In [82]:
len(y_test)

80057

In [83]:
y_train = list(map(int, y_train))
y_test = list(map(int, y_test))

In [94]:
batch_size = 300
train_dir = './cnn_input/pyplot_labelling/x_train'
test_dir = './cnn_input/pyplot_labelling/x_test'

In [95]:
import os
cpt = sum([len(files) for r, d, files in os.walk(train_dir)])
cpt

5853

In [96]:
x_train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(6, 6),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

Found 5850 files belonging to 2 classes.


In [97]:
x_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(6, 6),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

Found 2211 files belonging to 2 classes.


In [98]:
#### Building the Neural Net ####

In [99]:
def create_model():
    ## Initialize model.
    model = Sequential()

    ## 1st Layer
    model.add(Conv2D(34, kernel_size=3, activation='relu', input_shape=(6, 6, 1), name='conv2_1', groups=0))
    
    ## 2nd Layer
    model.add(Conv2D(32, kernel_size=3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='max_pool_1'))

    ## Adding 2  Dense Layers with sigmoid (for classification)
    model.add(Flatten())
    model.add(Dense(32, activation='relu', name='dense_1'))
    model.add(Dense(8, activation='relu', name='dense_2'))
    model.add(Dense(2, activation='sigmoid', name='dense_3'))
    
    ## Define loss function
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [100]:
FNN = create_model()

In [101]:
FNN.fit(x_train, epochs=100, batch_size=batch_size, shuffle=True, verbose=2)

Epoch 1/100
20/20 - 6s - loss: 2.3123 - accuracy: 0.5709
Epoch 2/100
20/20 - 6s - loss: 0.6975 - accuracy: 0.6311
Epoch 3/100
20/20 - 4s - loss: 0.6183 - accuracy: 0.6451
Epoch 4/100
20/20 - 4s - loss: 0.6161 - accuracy: 0.6474
Epoch 5/100
20/20 - 4s - loss: 0.6096 - accuracy: 0.6544
Epoch 6/100
20/20 - 4s - loss: 0.6047 - accuracy: 0.6508
Epoch 7/100
20/20 - 4s - loss: 0.6030 - accuracy: 0.6636
Epoch 8/100
20/20 - 4s - loss: 0.5986 - accuracy: 0.6581
Epoch 9/100
20/20 - 4s - loss: 0.5909 - accuracy: 0.6650
Epoch 10/100
20/20 - 4s - loss: 0.5884 - accuracy: 0.6709
Epoch 11/100
20/20 - 4s - loss: 0.5888 - accuracy: 0.6701
Epoch 12/100
20/20 - 4s - loss: 0.5835 - accuracy: 0.6740
Epoch 13/100
20/20 - 4s - loss: 0.5818 - accuracy: 0.6728
Epoch 14/100
20/20 - 4s - loss: 0.5823 - accuracy: 0.6747
Epoch 15/100
20/20 - 4s - loss: 0.5840 - accuracy: 0.6790
Epoch 16/100
20/20 - 5s - loss: 0.5828 - accuracy: 0.6778
Epoch 17/100
20/20 - 5s - loss: 0.5724 - accuracy: 0.6848
Epoch 18/100
20/20 - 6s

In [102]:
FNN.evaluate(x_test)

8/8 [==============================] - 0s 1ms/step - loss: 0.3962 - accuracy: 0.8322


[0.39617788791656494, 0.8322026133537292]

In [93]:
"""
Best so far:
23/23 [==============================] - 2s 83ms/step - loss: 0.3801 - accuracy: 0.8634
Batch Size: 100
Epochs = 100

23/23 [==============================] - 2s 86ms/step - loss: 0.6078 - accuracy: 0.6608
Batch Size: 100
Epochs = 10
"""

'\nBest so far:\nBatch Size: 100\nEpochs = 10\n'